# Tracer Correlation Factor computation for the <100>-<100> 90-degree dumbbell roto-translation mechanism in FCC systems.

In [1]:
import numpy as np
from onsager.crystal import Crystal
from onsager.crystalStars import zeroclean
from onsager.OnsagerCalc import dumbbellMediated
from onsager.crystal import DB_disp, DB_disp4, pureDBContainer, mixedDBContainer
import time

In [2]:
# make FCC lattice - all length units we use are in nanometers.
# Note that tracer correlation factors are independent of lattice parameters,
# as can be readily verified in the present calculations.

a0 = 1.0
a0cut = 1.01*a0/np.sqrt(2)  # The cutoff distance for dumbbell jumps. Setting to nearest neighbor and a
                            # small tolerance.

latt = np.array([[0., 0.5, 0.5], [0.5, 0., 0.5], [0.5, 0.5, 0.]]) * a0
FCC = Crystal(latt, [[np.array([0., 0., 0.])]], ["A"])

# Next, we set up the dumbbell containers.
# The "pdbcontainer" and "mdbcontainer" objects (see states.py) contain all information regarding pure and
# mixed dumbbells, including: The possible orientations (pdbcontainer.iorlist), the symmetry grouping
# of the dumbbells (pdbcontainer.symorlist), the group operations between them, and functions to generate
# pure and mixed dumbbell jump networks.

# We set up orientation vectors along <100> directions.
# Note that the orientation vector is a nominal vector. It helps in symmetry analysis and
# identificiation of jump types, but atomic displacements are only considered site-to-site in the
# transport coefficients. This simplification is allowed by the invariance principle of mass
# transport (Trinkle, 2018) without any errors in transport coefficients.

o = np.array([1.,0.,0.])*a0/4
famp0 = [o.copy()]  # multiple orientation families can be put in if desired, but we need only one.
family = [famp0]
pdbcontainer = pureDBContainer(FCC, 0, family)
mdbcontainer = mixedDBContainer(FCC, 0, family)

# Calculate the omega0 and omega2 jump networks.
# The inputs to both functions are as follows:
# The first input - the cutoff site-to-site distance for dumbbell jumps to be considered. We set this up to
# be the nearest neighbor distance.
# The next two inputs are cutoff distances for solute-solvent atoms and solvent-solvent atoms, such that
# if two atoms come within that distance of each other, then they are considered to collide (see collision.py)
jset0, jset2 = pdbcontainer.jumpnetwork(a0cut, 0.01, 0.01), mdbcontainer.jumpnetwork(a0cut, 0.01, 0.01)
print(FCC)

#Lattice:
  a1 = [0.  0.5 0.5]
  a2 = [0.5 0.  0.5]
  a3 = [0.5 0.5 0. ]
#Basis:
  (A) 0.0 = [0. 0. 0.]


In [3]:
# Modify jnet0
jnet0 = jset0[0]
# These contain jumps in the form of jump objects

jnet0_indexed = jset0[1] 
# These contain jumps in the form of ((i, j), dx) where "i" is the index assigned to the initial dumbbell state
# and j is the index assigned to the final dumbbell state.

# The 90-degree roto-translation jump is the jump that has the shortest total atomic dispalcements.
# Let's try to sort the jumps accordingly.

# We first get rid of the rotation jumps
z = np.zeros(3)
indices = []
for jt, jlist in enumerate(jnet0):
    
    # We'll skip on-site rotations for now and add them later
    if np.allclose(jnet0_indexed[jt][0][1], z):
        continue
    indices.append(jt)
    
def sortkey(entry):
    jmp = jnet0[entry][0]
    # Get the initial orientation vector.
    or1 = pdbcontainer.iorlist[jmp.state1.iorind][1]
    
    # Get the Final orientation vector.
    or2 = pdbcontainer.iorlist[jmp.state2.iorind][1]
    
    # Get the site-to-site displacements
    dx = DB_disp(pdbcontainer, jmp.state1, jmp.state2)
    
    # Get the nominal displacements along orientation vectors, which are only used to sort the jumps.
    # For the jump object, the quantity "c1" says whether the atom at the head (c1=1) or tail(c1=-1)
    # of the orientation vector is executing the jump, while the quantity "c2" says whether the jumping atom
    # ends up at the head (c2=1) or the tail (c2=-1) of the final orientation vector (see representations.py).
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3  # return the total displacement of the atoms.

# Sort the indices according to total displacements.
ind_sort = sorted(indices, key=sortkey)
#keep only the required index
ind_sort = [ind_sort[0]]

In [4]:
# Let's check if we got the correct jump

# For a dumbbell, the (i, or) indices correspond to (basis site, orientation vector) pairs.
# The corresponding values can be found at onsagercalculator.pdbcontainer.iorlist, which we print out
# in the next cell.

print(jnet0[ind_sort[0]][0])

Jump object:
Initial state:
	dumbbell : (i, or) index = 2, lattice vector = [0 0 0]
Final state:
	dumbbell : (i, or) index = 0, lattice vector = [-1  0  1]
Jumping from c1 = 1 to c2 = -1



In [5]:
# get the cartesian position of the final dumbbell with respect to the initial dumbbell
np.dot(FCC.lattice, jnet0[ind_sort[0]][0].state2.R)

array([ 0.5,  0. , -0.5])

In [6]:
pdbcontainer.iorlist

[(0, array([ 0.  ,  0.  , -0.25])),
 (0, array([ 0.  , -0.25,  0.  ])),
 (0, array([0.25, 0.  , 0.  ]))]

In [7]:
# take only the lowest displacement jump
# that is the jump we want.
jset0new = ([jnet0[ind_sort[0]]], [jnet0_indexed[ind_sort[0]]])

In [8]:
# Now, we modify the mixed dumbbell jumpnetwork to also give the lowest displacement jump
# in the same manner as the pure dumbbell jump network
jnet2 = jset2[0]
jnet2_indexed = jset2[1]
# Let's try to sort the jumps according to closest distance
# we don't want the rotational jumps as before.
z = np.zeros(3)
indices2 = []
for jt, jlist in enumerate(jnet2):
    if np.allclose(jnet2_indexed[jt][0][1], z):
        continue
    indices2.append(jt)    

def sortkey2(entry):
    jmp = jnet2[entry][0]
    or1 = mdbcontainer.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp(mdbcontainer, jmp.state1, jmp.state2)
    # c1 and c2 are always +1 for mixed dumbbell jumps.
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx + jmp.c2*or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(-jmp.c2*or2/2.)
    return dx1+dx2+dx3

ind_sort2 = sorted(indices2, key=sortkey2)
ind_sort2 = [ind_sort2[0]]

In [9]:
# check if we have the correct type of jump
print(jnet2[ind_sort2[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 4, lattice vector = [0 0 0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [-1  0  1]
	dumbbell : (i, or) index = 3, lattice vector = [-1  0  1]
Jumping from c1 = 1 to c2 = 1


In [10]:
for tup in mdbcontainer.iorlist:
    print(tup)

(0, array([ 0.  ,  0.  , -0.25]))
(0, array([ 0.  , -0.25,  0.  ]))
(0, array([0.  , 0.25, 0.  ]))
(0, array([0.  , 0.  , 0.25]))
(0, array([0.25, 0.  , 0.  ]))
(0, array([-0.25,  0.  ,  0.  ]))


In [11]:
# take only the lowest displacement jump
# that is the jump we want.
jset2new = ([jnet2[ind_sort2[0]]], [jnet2_indexed[ind_sort2[0]]])

In [12]:
# Next, make an initial onsager calculator
# All possible omega4-3 jumps will first be found, the cutoff distances of which are indicated as follows:
# a0cut : cutoff for site-to-site atomic displacement during omega4-3 jumps.
# 0.01 : distance of closest approach during omega4-3 jumps. Input twice for solute-solute and solute-solvent
# atom pairs.
# Nthermo - the thermodynamic shell - nominally one nearest neighbor in the present case.
start = time.time()
onsagercalculator = dumbbellMediated(pdbcontainer, mdbcontainer, jset0new, jset2new, a0cut,
                                     0.01, 0.01, 0.01, NGFmax=4, Nthermo=1)
print("onsager calculator initiation time = {}".format(time.time() - start))

initializing thermo
initializing kin
generating thermodynamic shell
built shell 1: time - 0.006982326507568359
grouped states by symmetry: 0.10205364227294922
built mixed dumbbell stars: 0.0004031658172607422
built jtags2: 3.910064697265625e-05
built mixed indexed star: 0.002238750457763672
building star2symlist : 6.341934204101562e-05
building bare, mixed index dicts : 0.000152587890625
thermodynamic shell generated: 0.16315650939941406
Total number of states in Thermodynamic Shell - 39, 6
generating kinetic shell
built shell 1: time - 0.00654911994934082
built shell 2: time - 0.32631850242614746
grouped states by symmetry: 1.0448944568634033
built mixed dumbbell stars: 0.0004258155822753906
built jtags2: 4.029273986816406e-05
built mixed indexed star: 0.0021882057189941406
building star2symlist : 0.00012111663818359375
building bare, mixed index dicts : 0.00020813941955566406
Kinetic shell generated: 2.354548215866089
Total number of states in Kinetic Shell - 165, 6
generating kineti

In [13]:
# Next, we get the omega43 jumps computed internally in the onsager calculator,
# and extract the shortest displacement jumps in the same manner as the omega0 and omega2 jumps.
jnet43 = onsagercalculator.jnet43
jnet43_indexed = onsagercalculator.jnet43_indexed
# Let's try to sort the jumps according to closest distance
z = np.zeros(3)
indices43 = []
for jt, jlist in enumerate(jnet43):
    if np.allclose(jnet43_indexed[jt][0][1], z):
        continue
    indices43.append(jt)

def sortkey43(entry):
    jmp = jnet43[entry][0] # This is an omega4 jump
    if not jmp.c2 == -1:
        print(c2)
    or1 = pdbcontainer.iorlist[jmp.state1.db.iorind][1]
    or2 = mdbcontainer.iorlist[jmp.state2.db.iorind][1]
    dx = DB_disp4(pdbcontainer, mdbcontainer, jmp.state1, jmp.state2)
    # remember that c2 is -1 for an omega4 jump
    dx1 = np.linalg.norm(jmp.c1*or1/2.)
    dx2 = np.linalg.norm(dx - or2/2. - jmp.c1*or1/2.)
    dx3 = np.linalg.norm(jmp.c2*or2/2.)
    return dx1+dx2+dx3

ind_sort43 = sorted(indices43, key=sortkey43)[:1]

In [14]:
# check if we have the correct jump
print(jnet43[ind_sort43[0]][0])

Jump object:
Initial state:
	Solute loctation:basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 1, lattice vector = [ 1 -1  0]
Final state:
	Solute loctation :basis index = 0, lattice vector = [0 0 0]
	dumbbell : (i, or) index = 4, lattice vector = [0 0 0]
Jumping from c1 = 1 to c2 = -1


In [15]:
pdbcontainer.iorlist

[(0, array([ 0.  ,  0.  , -0.25])),
 (0, array([ 0.  , -0.25,  0.  ])),
 (0, array([0.25, 0.  , 0.  ]))]

In [16]:
mdbcontainer.iorlist

[(0, array([ 0.  ,  0.  , -0.25])),
 (0, array([ 0.  , -0.25,  0.  ])),
 (0, array([0.  , 0.25, 0.  ])),
 (0, array([0.  , 0.  , 0.25])),
 (0, array([0.25, 0.  , 0.  ])),
 (0, array([-0.25,  0.  ,  0.  ]))]

### Now that we have the correct jump, we need to reconstruct the omega43 jump list, the bias expansions and the rate expansions.

In [17]:
# reconstruct the omega43 jump network to include only the 90-degree roto-translational jumps we have just
# filtered out.
onsagercalculator.regenerate43(ind_sort43)

In [18]:
# Next, we need to provide some nominal thermodynamic data for tracer correlation factors.
# All the energies of the mixed and pure dumbbells will be the same,
# All the jump rates will be the same
    # For each of omega0, omega2 and omega43 jumps, set their barriers to zero.
    # All omega1 rates will be the same as the above rate.
# The "solute" energies will be zero since we are dealing with a chemically identical tracer.
# All interaction energies will be zero.

# 1a. Energies and pre-factors
kT = 1

predb0, enedb0 = np.ones(len(pdbcontainer.symorlist)), \
                 np.random.rand(len(pdbcontainer.symorlist))

preS, eneS = np.ones(len(FCC.sitelist(onsagercalculator.vkinetic.starset.chem))), \
             np.zeros(len(FCC.sitelist(onsagercalculator.vkinetic.starset.chem)))

# These are the interaction or the excess energies and pre-factors for solutes and dumbbells.
# The energies will all be zero.
preSdb, eneSdb = np.ones(onsagercalculator.thermo.mixedstartindex), \
                 np.zeros(onsagercalculator.thermo.mixedstartindex)

predb2, enedb2 = predb0.copy(), enedb0.copy()

preT0, eneT0 = np.ones(len(onsagercalculator.vkinetic.starset.jnet0)), np.zeros(len(onsagercalculator.jnet0))
preT2, eneT2 = preT0.copy(), eneT0.copy()

# Next, we set the omega1 and omega0 migration barrier to be the same, by going through
# the array "om1types" which stores which pure dumbbell (omega0) jump a complex-complex jump corresponds to.
preT1, eneT1 = np.ones(len(onsagercalculator.jnet1)), np.array([eneT0[onsagercalculator.om1types[jt]] for jt in
                                                                range(len(onsagercalculator.jnet1))])

preT43, eneT43 = preT0.copy(), eneT0.copy()

In [19]:
# Next, convert the energies and pre-factors to (1/kT*Free Energy)
bFdb0, bFdb2, bFS, bFSdb, bFT0, bFT1, bFT2, bFT3, bFT4 = \
    onsagercalculator.preene2betafree(kT, predb0, enedb0, preS, eneS, preSdb, eneSdb, predb2, enedb2,
                                           preT0, eneT0, preT2, eneT2, preT1, eneT1, preT43, eneT43)

In [20]:
# get the transport coefficients, Green's functions (total and non-local), state probabilities and rates
# from the onsagercalculator.L_ij function

# L_uc_aa, L_c_aa : uncorrelated and correlated tracer-tracer (or solute-solute) transport coefficient
#                   contributions                    

# L0bb : bare dumbbell transport coefficient.
# L_uc_bb, L_c_bb : uncorrelated and correlated solvent-solvent transport coefficient
#                   contributions due to solute interactions.

# L_uc_ab, L_c_ab : uncorrelated and correlated solute-solvent transport coefficient
#                   contributions.

L0bb, (L_uc_aa, L_c_aa), (L_uc_bb, L_c_bb), (L_uc_ab, L_c_ab)=\
onsagercalculator.L_ij(bFdb0, bFT0, bFdb2, bFT2, bFS, bFSdb, bFT1, bFT3, bFT4)

In [21]:
L_aa = L_uc_aa + L_c_aa

In [22]:
L_ab = L_uc_ab + L_c_ab

In [23]:
L_aa[0][0]/L_ab[0][0]

0.4395080938304338

In [24]:
# Check that this is the same as the ratio of the tracer-tracer transport coefficient to its uncorrelated
# part
L_aa[0][0]/(L_uc_aa[0][0])

0.4395080938304338

Note that in order to keep the two definitions of the transport coefficient consistent, we need to only consider site-to-site displacements. For if orientational displacements (positions of atoms in a dumbbell away from lattice sites) are considered, then the uncorrelated part of the solute-solute transport will not be invariant with those displacements, even though the total transport coefficient will be (See Trinkle, Phys. Rev. Lett., 2018 for the invariance principle of mass transport).